In [1]:
import datetime
import numpy
import netCDF4
from osgeo import gdal, osr
import ogr
import os
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing as mp
import collections
import logging
import time

logging.basicConfig(level=logging.DEBUG, filename=r'P:\projects\SILO\siloLog.txt')

#example
#start = '2015-01-01'
#end = '2015-03-23'
#sd = dt.strptime(start,'%Y-%m-%d')
#ed = dt.strptime(end,'%Y-%m-%d')
#delta = ed - sd
#for i in range(delta.days+1):
#    dict[sd + td(days=i)] = <something>

reqStart = '1980-01-01'
#reqEnd = '1981-12-31'
reqEnd = '1980-12-31'

"""
inputGeoCoordsFile = r'P:\projects\SILO\Wes\MNDB_geocoords.csv'
outputDir = r'P:\projects\SILO\Wes\testOut2'
siloNetCDFDir = r'P:\projects\SILO\Wes\mwet'

varNameInSilo = 'et_morton_wet'
varForColName = 'pet'

"""
inputGeoCoordsFile = r'C:\DDrive\Maria\CatchModCSVs\WT_RC8_30m_climcoords.csv'
outputDir = r'C:\DDrive\Maria\ClimateOuts'
#siloNetCDFDir = r'C:\DDrive\Maria\netCDFs\rainfall'
siloNetCDFDir = r'P:\projects\SILO\Wes\mwet'

varNameInSilo = 'et_morton_wet'
varForColName = 'pet'



#Col names for GeoCoords CSV
catGrpName = 'CATCHGROUP'
latName = 'SILO_LAT'
longName = 'SILO_LONG'
countName = 'COUNT'

In [2]:
#Base output dir
if not os.path.exists(outputDir):
    os.makedirs(outputDir)
    print("Made dir: " + outputDir)


In [3]:
#Read in GeoCoords
GeoCoordsVals = {}

geoCoordsDF = pd.read_csv(inputGeoCoordsFile)
#for index, row in geoCoordsDF.iterrows():
#    if not row[catGrpName] in GeoCoordsVals:
#        #print('Adding: ' + str(row[catFileRasID]) + ' and ' + str(row[catFileCatName]))
#        GeoCoordsVals[row[catGrpName]] = row[catFileCatName]

allCats = geoCoordsDF[catGrpName].unique().tolist()
colNames = [varForColName + ' for ' + cat for cat in allCats]
print(len(allCats))

517


In [4]:

sd = datetime.datetime.strptime(reqStart,'%Y-%m-%d')
ed = datetime.datetime.strptime(reqEnd,'%Y-%m-%d')
##delta = ed - sd

sYear = sd.year
eYear = ed.year

allYears = []
for y in range(sYear, eYear + 1):
    if not y in allYears:
        allYears.append(y)

#date_today = datetime.now()
#days = pd.date_range(date_today, date_today + timedelta(7), freq='D')
date_rng = pd.date_range(sd, ed, freq='D')
#print(str(type(days[0])))
daysAsList = date_rng.tolist()
#mainDF = pd.DataFrame({'Date': days})
mainDF = pd.DataFrame(date_rng, columns=['tempDate'])
#mainDF = mainDF.set_index('Date')

#Getting date as index????
mainDF['Date'] = pd.to_datetime(mainDF['tempDate'])
mainDF = mainDF.set_index('Date')
mainDF.drop(['tempDate'], axis=1, inplace=True)


for col in colNames:
    mainDF[col] = ''

resultsDict = {}
#for d in daysAsList:
#    resultsDict[d] = {}



mainDF.head()
#print(mainDF)

,pet for SC #4,pet for SC #35,pet for SC #1,pet for SC #5,pet for SC #34,pet for SC #90488,pet for SC #2,pet for SC #3,pet for SC #9,pet for SC #33,...,pet for SC #381,pet for SC #379,pet for SC #380,pet for SC #469,pet for SC #376,pet for SC #386,pet for SC #377,pet for SC #378,pet for SC #387,pet for SC #388
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,,,,,,,,,,,...,,,,,,,,,,
1980-01-02,,,,,,,,,,,...,,,,,,,,,,
1980-01-03,,,,,,,,,,,...,,,,,,,,,,
1980-01-04,,,,,,,,,,,...,,,,,,,,,,
1980-01-05,,,,,,,,,,,...,,,,,,,,,,


In [5]:
#class to contain a raster band
DailyRasterBand = collections.namedtuple('DailyRasterBand', [
    'dateOfRaster',
    'RasterArray',
    'geoTransObject',
    'scaleFactor',
    'noDataVal'
])

In [6]:
def processBand(number, allBands):

    result = {}
    srcband = allBands[number]

    #logging.info('Checking this item: ' + str(number) + ' is a ' + str(type(srcband.RasterBand))
    #             + 'of date ' + srcband.dateOfRaster)
   
    #scaleFactor = 1.0
        
    #sf = srcband.RasterBand.GetMetadataItem(varNameInSilo + '_scale_factor')
    #nd = srcband.RasterBand.GetMetadataItem(varNameInSilo + '__FillValue')
    #nd2 = srcband.RasterBand.GetNoDataValue()

        
    scaleFactor = float(srcband.scaleFactor)
    rawNoData = float(srcband.noDataVal)
    
    fakeSiloGT = srcband.geoTransObject
    #siloArray = srcband.RasterBand.ReadAsArray()
    ##just make sure there's no gaps? Not necessary?
    ##siloMask = numpy.ma.masked_where(siloArray == nd, siloArray)
    
    print(srcband.dateOfRaster + ' ' + str(len(srcband.RasterArray)))
    #logging.info('Checking this array: ' + str(siloArray))
        
    #print(dateForFileName + ': ' + str(siloMask[500,500] * scaleFactor))
    
    ##NOTE: Updated GDAL means we don't need to invert anymore
    #Invert Y-axis values...
    #Really important
    #invertArray = srcband.RasterArray[::-1,:]
        
    for thisCat in allCats:
            
        colName = varForColName + ' for ' + thisCat
        catTodaySum = 0.0
        catTodayCount = 0.0
            
        catDF = geoCoordsDF.loc[geoCoordsDF[catGrpName] == thisCat]
            
        for index, row in catDF.iterrows():
                
            longVal = float(row[longName])
            latVal = float(row[latName])
            countVal = int(row[countName])
                
            geoXcell = int((longVal - fakeSiloGT[0]) / fakeSiloGT[1]) #x pixel
            geoYcell = int((latVal - fakeSiloGT[3]) / fakeSiloGT[5]) #y pixel
                
            ##NOTE: Updated GDAL means we don't need to invert anymore
            #catTodaySum += (invertArray[geoYcell,geoXcell] * scaleFactor) * countVal
            catTodaySum += (srcband.RasterArray[geoYcell,geoXcell] * scaleFactor) * countVal
            catTodayCount += countVal
        
        
        #Calc average
        avVal = 0
        if catTodayCount > 0:
            avVal = catTodaySum / catTodayCount
            
        result.update({thisCat:avVal})
    
    #Try and jam into dataframe
    #for cat,val in result.items():
    #    colName = varForColName + ' for ' + cat
    #    mainDF[srcband.dateOfRaster][colName] = val
    
    resultsDict[srcband.dateOfRaster] = result
    
    return
    #return {
    #    'name': item.name,
    #    'age': 2017 - item.born
    #}

In [7]:
#load the data
#print(allYears)
print('started at: ' + str(datetime.datetime.now()))
data = (['a', '2'], ['b', '4'], ['c', '6'], ['d', '8'],['e', '1'], ['f', '3'], ['g', '5'], ['h', '7'])
numbers = [5, 10, 15, 20, 25]
procs = []

for thisYear in allYears:
    
    siloFile = os.path.join(siloNetCDFDir, str(thisYear) + '.' + varNameInSilo + '.nc')
    print('Processing: ' + siloFile)
    #print(type(thisYear))
    
    #Need the gdal version of silo to give us a few params, but will close of netCDF version pronto...
    siloDS = gdal.Open(siloFile)
    
    #Updated GDAL means we don't need this anymore
    """
    ##Need this to calc pixel size as well as work out order of y-axis
    netCDFData = netCDF4.Dataset(siloFile, 'r')

    #Grab the info we need to build GeoTransform object
    #Checking for order of y-axis coordinates, would be needed if writing raster form Numpy array
    
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    first_lat = lats[:1]
    last_lat = lats[len(lats)-1:]
    #If first_lat is LESS THAN last_lat, numpy array would be written to raster upside down...
    
    first_lon = lons[:1]
    last_lon = lons[len(lons)-1:]
    
    colCount = siloDS.RasterXSize
    rowCount = siloDS.RasterYSize
    
    pixWidth = (last_lon - first_lon) / (siloDS.RasterXSize - 1)
    pixHeight = (last_lat - first_lat) / (siloDS.RasterYSize - 1)

    if first_lat < last_lat:
        #adjust for our use
        pixHeight *= -1
    
    
    siloLeft = min(lons) - (0.5 * pixWidth)#adjusting to get cell edge, not centre
    siloTop = max(lats) - (0.5 * pixHeight)#adjusting to get cell edge, not centre
    
    netCDFData.close()
    """
    
    #fakeSiloGT = [siloLeft, pixWidth, 0, siloTop, 0, pixHeight]
    #The GDal dataset should have the pixel height etc already, maybe my old GDAL version couldn't read it???
    fakeSiloGT = siloDS.GetGeoTransform()
    
    #Jan 1
    siloDStart = datetime.datetime(year=thisYear, month=1, day=1)
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    
    allbands = []
    for band in range(0, siloDS.RasterCount):
        
        dateNow = siloDStart + datetime.timedelta(days=band)
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        dateTimeStampKey = str(dateNow.year) + '-' + str(dateNow.month).zfill(2) + '-' + str(dateNow.day).zfill(2)
        
        if not dateNow in daysAsList:
            continue
        
        #print("Getting: " + dateTimeStampKey + " from Idx: " + str(band))
        theBand = siloDS.GetRasterBand(band + 1)
        
        #Updated GDAL means we don't need var name
        #theSF = theBand.GetMetadataItem(varNameInSilo + '_scale_factor')
        #theND = theBand.GetMetadataItem(varNameInSilo + '__FillValue')
        theSF = theBand.GetMetadataItem('scale_factor')
        theND = theBand.GetMetadataItem('_FillValue')
        
        scaleFactor = float(theSF)
        rawNoData = float(theND)

        
        allbands.append(DailyRasterBand(dateOfRaster=dateTimeStampKey,
                                        RasterArray=theBand.ReadAsArray(), geoTransObject=fakeSiloGT,
                                       scaleFactor=scaleFactor, noDataVal=rawNoData))
    
    num_cores = mp.cpu_count()
    procList = list(range(1, siloDS.RasterCount + 1))
    #Parallel(n_jobs=num_cores)(delayed(doubler) for i in numbers)#in enumerate(numbers)#, siloDS=siloDS
    #Parallel(n_jobs=num_cores, require='sharedmem')(delayed(doubler)(i) for i in range(5))
    Parallel(n_jobs=num_cores, require='sharedmem')(delayed(processBand)(i, allbands) for i in range(0, len(allbands)))
    #print(ff)
    #print(shared_set)
    
    #pool = mp.Pool(mp.cpu_count())
    #print(str(mp.cpu_count()))
    ##results = [pool.apply(grabDailyVals2, args=(row)) for row in procList]
    #pool.close() 
    
#mainDF.head()
print('finished at: ' + str(datetime.datetime.now()))

started at: 2019-11-04 11:55:16.566218
Processing: P:\projects\SILO\Wes\mwet\1980.et_morton_wet.nc
1980-01-01 6811980-01-02 6811980-01-03 681
1980-01-04 681

1980-01-05 681
1980-01-06 681
1980-01-07 6811980-01-08 681


1980-01-09 681
1980-01-10 6811980-01-11 681
1980-01-12 681
1980-01-13 681

1980-01-14 681
1980-01-15 6811980-01-16 681
1980-01-17 681

1980-01-18 681
1980-01-19 681
1980-01-20 681
1980-01-21 681
1980-01-22 681
1980-01-23 681
1980-01-24 6811980-01-25 681

1980-01-26 6811980-01-27 681

1980-01-28 681
1980-01-29 6811980-01-30 681
1980-01-31 681

1980-02-01 681
1980-02-02 6811980-02-03 681

1980-02-04 681
1980-02-05 681
1980-02-06 681
1980-02-07 681
1980-02-08 6811980-02-09 681

1980-02-10 681
1980-02-11 681
1980-02-12 681
1980-02-13 681
1980-02-14 681
1980-02-15 681
1980-02-16 681
1980-02-17 681
1980-02-18 6811980-02-19 681

1980-02-20 681
1980-02-21 681
1980-02-22 681
1980-02-23 6811980-02-24 681

1980-02-25 681
1980-02-26 681
1980-02-27 6811980-02-28 681

1980-02-29 68119

In [8]:
for t,data in resultsDict.items():
    for col,val in data.items():
        colName = varForColName + ' for ' + col
        mainDF.loc[t][colName] = val

In [9]:
mainDF.head()

,rainfall for SC #4,rainfall for SC #35,rainfall for SC #1,rainfall for SC #5,rainfall for SC #34,rainfall for SC #90488,rainfall for SC #2,rainfall for SC #3,rainfall for SC #9,rainfall for SC #33,...,rainfall for SC #381,rainfall for SC #379,rainfall for SC #380,rainfall for SC #469,rainfall for SC #376,rainfall for SC #386,rainfall for SC #377,rainfall for SC #378,rainfall for SC #387,rainfall for SC #388
Date,,,,,,,,,,,,,,,,,,,,,
1960-01-01,0.00631303,7.34655e-05,0.0668382,0,0.0735395,-7.17476,0.0181065,0,0.0070245,0.143236,...,1.27569,1.37638,1.5671,1.34051,0.993975,1.58754,1.13403,1,0.910901,0.480868
1960-01-02,2.8851,0.96806,4.46735,2.57483,0.198267,-5.60524,3.24048,4.19509,1.05972,0.287633,...,2.65138,3.02003,3.30279,2.88964,2.38393,3.42326,2.50463,2.4,2.02148,1.05076
1960-01-03,11.426,10.2143,8.9671,9.99247,7.59816,-0.0736998,8.41281,8.80532,9.37237,7.62152,...,0.391897,0.499009,0.534758,0.461571,0.397992,0.542261,0.398244,0.4,0.344292,0.179142
1960-01-04,1.17005,1.86271,1.06632,1.77886,1.93212,-6.09189,1.4161,1.20123,2.80443,2.71263,...,0.0162055,0.0534673,0.00483271,0,0.1,0.0030233,0.00479341,0.1,0.0804873,0.11788
1960-01-05,0,0,0,0,0,-7.20739,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
mainDF.to_csv(r'C:\DDrive\Maria\WTrain1960.csv', index=True)